In [1]:
import altair as alt
import pandas as pd
import numpy as np
from scipy.stats import beta

In [2]:
z95 = 1.959963984540054
results = pd.read_csv('sem_results.csv', index_col=0)
method_names = results['methodName'].unique()
timeseries_lengths = results['timeSeriesLength'].unique()


#calculate which experiments had the true mean within the estimatimed_mean +/- abs(SEM)
###If using the frequentist CI below, calculate a rate instead of a count:
#results['rate'] = results.apply(lambda row: float(np.abs(row['estMean']) < z95*row['SEM'])*0.01, axis=1)
results['rate'] = results.apply(lambda row: 0.01*float(np.sign(np.array([row['mean_low'], row['mean_high']])).sum()==0), axis=1)

##Calculate size of the CI:
results['ci_size'] = results.apply(lambda row: row['mean_high']-row['mean_low'], axis=1)

#group by all the experimental conditions, sum the number of correct SEMs per condition,
#then flatten into long format with reset index:
data = pd.DataFrame(results.groupby(['methodName', 'timeSeriesLength', 'trueRho'])['rate'].sum()).reset_index()

#calculate confidence intervals for proportions:
##This uses standard binomial confidence interval:
##(equivalent to from statsmodels.stats.proportion import proportion_confint)
data['confInt'] = data.apply(lambda row: 1.96*np.sqrt(row['rate']*(1-row['rate'])/100), axis=1).fillna(0)

##Alternatively, can calculate the CI using a bayesian conjugate prior (beta distribution)
##I am using a beta(1,1) prior, hence the '+1's. Statsmodels has the Jeffreys interval which uses (0.5,0.5)
data['confIntLow'] = data.apply(lambda row: beta.ppf(0.025, 100*row['rate']+1, 100-100*row['rate']+1), axis=1)
data['confIntHigh'] = data.apply(lambda row: beta.ppf(0.975, 100*row['rate']+1, 100-100*row['rate']+1), axis=1)

In [3]:
#This is simply to avoid formatting the faceted plot later:
data['trueRho'] =data.apply(lambda row: 'ρ='+str(row['trueRho']),axis=1)

In [4]:
#see how poorly the automated block averaging estimates SEM particularly for low rho cases:
#results.groupby(['methodName', 'trueRho', 'timeSeriesLength'])['SEM'].std().unstack()

#Turn SEM into an estimate of the correlation time:
#results


# Static version:

In [5]:
# the base chart
base = alt.Chart(data).transform_calculate(
    x_jittered = '0.15*random()*datum.timeSeriesLength+datum.timeSeriesLength',
     #ymin="datum.rate-datum.confInt",
     #ymax="datum.rate+datum.confInt",
    ymin = "datum.confIntLow",
    ymax = "datum.confIntHigh",
    goal='0.95')



#generate the scatter points:
points = base.mark_point(filled=True).encode(
    x=alt.X('x_jittered:Q', scale=alt.Scale(type='log'), title='Length of Timeseries'),
    y=alt.Y('rate:Q', scale=alt.Scale(domain=[0,1.04]), title='Rate of correct SEM'),
    size=alt.value(80),
    color=alt.Color('methodName', legend=alt.Legend(title="SEM method")))

#generate the scatter points:
line = base.mark_line().encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('rate:Q'),
    color='methodName')

# scale=alt.Scale(type='log', domain=[0.001, 1000]),
#           axis=alt.Axis(tickCount=5)
    
#generate the 95% mark:
rule = base.mark_rule(color='black').encode(
    alt.Y('goal:Q'))

# generate the error bars:
# errorbars = base.mark_errorbar().encode(
#     alt.X("x_jittered:Q"),
#     alt.Y("ymin:Q", title=''),
#     alt.Y2("ymax:Q"),
#     color='methodName')

errorbars = base.mark_rule(strokeWidth=3).encode(
    alt.X("x_jittered:Q"),
    alt.Y("ymin:Q", title=''),
    alt.Y2("ymax:Q"),
    color='methodName'
)


selection = alt.selection_single(fields=['symbol']);

chart = alt.layer(
    
  errorbars,
  points,
  line,
  rule,
    
).properties(
    width=250,
    height=200
    ).add_selection(selection).facet(facet=alt.Facet('trueRho:N', 
                                                     title='Autocorrelation parameter (ρ)'), columns=3)


chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


chart

alt.FacetChart(...)

In [8]:
chart.save('hi.svg')

In [9]:
!npm install canvas

        ░░░░░░⸩ ⠏ build:delegates: sill linkStuff delegates@1.0.0 has /Usem
> canvas@2.6.1 install /Users/lmar3213/Documents/GitHub/estimating_sem_timeseries/node_modules/canvas
> node-pre-gyp install --fallback-to-build

node-pre-gyp WARN Using needle for node-pre-gyp https download 
node-pre-gyp WARN Pre-built binaries not installable for canvas@2.6.1 and node@13.13.0 (node-v79 ABI, unknown) (falling back to source compile with node-gyp) 
node-pre-gyp WARN Hit error Remote end closed socket abruptly. 
node-pre-gyp WARN Pre-built binaries not installable for canvas@2.6.1 and node@13.13.0 (node-v79 ABI, unknown) (falling back to source compile with node-gyp) 
node-pre-gyp WARN Hit error ENOENT: Cannot cd into '/Users/lmar3213/Documents/GitHub/estimating_sem_timeseries/node_modules/canvas/build/Release' 
gyp: /Users/lmar3213/Documents/GitHub/estimating_sem_timeseries/node_modules/canvas/build/config.gypi not found (cwd: /Users/lmar3213/Documents/GitHub/estimating_sem_timeseries/node_mod

# Interactive version:


In [37]:


# the base chart
base = alt.Chart(data).transform_calculate(
    x_jittered = '0.15*random()*datum.timeSeriesLength+datum.timeSeriesLength',
    #ymin="datum.rate-datum.confInt",
    #ymax="datum.rate+datum.confInt",
    ymin = "datum.confIntLow",
    ymax = "datum.confIntHigh",
    goal='0.95')


selector = alt.selection_single(
        fields=['methodName'], 
        empty='all',
    bind='legend'
    )

#generate the scatter points:
points = base.mark_point(filled=True).add_selection(selector).encode(
    x=alt.X('x_jittered:Q', scale=alt.Scale(type='log'), title='Length of Timeseries'),
    y=alt.Y('rate:Q', scale=alt.Scale(domain=[0,1.04]), title='Rate of correct SEM'),
    size=alt.value(80),
    color=alt.condition(selector, 'methodName:N', alt.value('lightgrey'),legend=alt.Legend(title='SEM Method-Click to highlight!')),
    tooltip=['methodName:N'],)
    #color='methodName')
    
    
# alt.Chart(iris).mark_point().encode(
#     x='petalWidth',
#     y='petalLength',    
#     color=alt.condition(click,
#                         'species:N', alt.value('lightgray'), 
#                         scale=palette,
#                         legend=None)
    
selector = alt.selection_single(
        fields=['methodName'], 
        empty='all',
    bind='legend'
    )
    
#generate the lines:
line = base.mark_line().add_selection(selector).encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('rate:Q'),
    color=alt.condition(selector, 'methodName:N', alt.value('lightgrey')))
    #color='methodName')

# #generate the 95% mark:
# rule = base.mark_rule(color='black').encode(
#     alt.Y('goal:Q'))

selector = alt.selection_single(
        fields=['methodName'], 
        empty='all',
    bind='legend'
    )

errorbars = base.mark_rule(strokeWidth=3).add_selection(selector).encode(
    alt.X("x_jittered:Q"),
    alt.Y("ymin:Q", title=''),
    alt.Y2("ymax:Q"),
    color=alt.condition(selector, 'methodName:N', alt.value('lightgrey')))
    #color='methodName')

In [38]:
chart = alt.layer(
    
  errorbars,
  points,
  line,
  rule,
    
).properties(
    width=250,
    height=200
).facet(facet=alt.Facet('trueRho:N', title='Autocorrelation parameter (ρ)'), columns=3)


In [39]:
chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


In [40]:
chart.interactive()

alt.FacetChart(...)

# Static plot of CI Size
First plot the mean and CI of the estimated extents using a symlog scale:

In [220]:
data2 = pd.DataFrame(results.groupby(['methodName', 'timeSeriesLength', 'trueRho'])['ci_size'].mean()).reset_index()
data2['std'] = results.groupby(['methodName', 'timeSeriesLength', 'trueRho'])['ci_size'].std().values
data2['confIntLow'] = data2.apply(lambda row: row['ci_size']-row['std'], axis=1)
data2['confIntHigh'] = data2.apply(lambda row: row['ci_size']+row['std'], axis=1)


In [221]:
# the base chart
base = alt.Chart(data2).transform_calculate(
    x_jittered = '0.05*random()*datum.timeSeriesLength+datum.timeSeriesLength',
    ymin = "datum.confIntLow",#"(1.95996*datum.std / 100)",
    ymax = "datum.confIntHigh",#"(1.95996*datum.std / 100)",
)


#generate the scatter points:
points = base.mark_point(filled=True).encode(
    x=alt.X('x_jittered:Q',scale=alt.Scale(type='log'),title='Length of Timeseries'),
    y=alt.Y('ci_size:Q',scale=alt.Scale(type='symlog'),title='Size of the CI'),
    size=alt.value(80),
    color=alt.Color('methodName', legend=alt.Legend(title="SEM method")))

#scale=alt.Scale(type='log')
#generate the scatter points:
line = base.mark_line().encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('ci_size:Q'),
    color='methodName')

    
##generate the 95% mark:
#rule = base.mark_rule(color='black').encode(
#    alt.Y('goal:Q'))

# #generate the error bars:
# errorbars = base.mark_errorbar().encode(
#     alt.X("x_jittered:Q"),
#     alt.Y("ymin:Q", title=''),
#     alt.Y2("ymax:Q"),
#     color='methodName')


# errorbars = base.mark_errorbar().encode(
#    x=alt.X("x_jittered:Q"),
#    y = alt.Y("confIntLow:Q", title=''),
#    y2=alt.Y2("confIntHigh:Q"),
#    color='methodName'
# )
errorbars = base.mark_rule(strokeWidth=3).encode(
   alt.X("x_jittered:Q"),
   alt.Y("ymin:Q", title=''),
   alt.Y2("ymax:Q"),
   color='methodName'
)

chart = alt.layer(
    
  errorbars,
  points,
  line
    
).properties(
    width=250,
    height=200
    ).facet(facet=alt.Facet('trueRho:N',title='Autocorrelation parameter (ρ)'), columns=3)


chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


chart.interactive()

alt.FacetChart(...)

# However
it might be more appropriate just to plot the means, and/or the medians:

In [224]:
data2 = pd.DataFrame(results.groupby(['methodName', 'timeSeriesLength', 'trueRho'])['ci_size'].mean()).reset_index()

# the base chart
base = alt.Chart(data2).transform_calculate(
    x_jittered = '0.05*random()*datum.timeSeriesLength+datum.timeSeriesLength',
    ymin = "datum.confIntLow",#"(1.95996*datum.std / 100)",
    ymax = "datum.confIntHigh",#"(1.95996*datum.std / 100)",
)


#generate the scatter points:
points = base.mark_point(filled=True).encode(
    x=alt.X('x_jittered:Q',scale=alt.Scale(type='log'),title='Length of Timeseries'),
    y=alt.Y('ci_size:Q',scale=alt.Scale(type='log'),title='Mean size of the CI'),
    size=alt.value(80),
    color=alt.Color('methodName', legend=alt.Legend(title="SEM method")))

#generate the scatter points:
line = base.mark_line().encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('ci_size:Q'),
    color='methodName')

chart = alt.layer(
    
  points,
  line
    
).properties(
    width=250,
    height=200
    ).facet(facet=alt.Facet('trueRho:N',title='Autocorrelation parameter (ρ)'), columns=3)


chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


chart.interactive()

alt.FacetChart(...)

In [225]:
data2 = pd.DataFrame(results.groupby(['methodName', 'timeSeriesLength', 'trueRho'])['ci_size'].median()).reset_index()

# the base chart
base = alt.Chart(data2).transform_calculate(
    x_jittered = '0.05*random()*datum.timeSeriesLength+datum.timeSeriesLength',
    ymin = "datum.confIntLow",#"(1.95996*datum.std / 100)",
    ymax = "datum.confIntHigh",#"(1.95996*datum.std / 100)",
)


#generate the scatter points:
points = base.mark_point(filled=True).encode(
    x=alt.X('x_jittered:Q',scale=alt.Scale(type='log'),title='Length of Timeseries'),
    y=alt.Y('ci_size:Q',scale=alt.Scale(type='log'),title='Median size of the CI'),
    size=alt.value(80),
    color=alt.Color('methodName', legend=alt.Legend(title="SEM method")))

#generate the scatter points:
line = base.mark_line().encode(
    x=alt.X('x_jittered:Q'),
    y=alt.Y('ci_size:Q'),
    color='methodName')

chart = alt.layer(
    
  points,
  line
    
).properties(
    width=250,
    height=200
    ).facet(facet=alt.Facet('trueRho:N',title='Autocorrelation parameter (ρ)'), columns=3)


chart = chart.configure_header(titleColor='darkred',
                       titleFontSize=16,
                      labelColor='darkred',
                      labelFontSize=14)

chart = chart.configure_legend(
    strokeColor='gray',
    fillColor='#EEEEEE',
    padding=10,
    cornerRadius=10,
    orient='top'
)


chart.interactive()

alt.FacetChart(...)